In [1]:
quietly = function(x) {
    return(suppressWarnings(suppressMessages(x)))
}

In [2]:
quietly(library(GEOquery))
quietly(library(rjson))
quietly(library(dplyr))
quietly(library(tidyr))
quietly(library(tibble))
quietly(library(stringr))
quietly(library(limma))
quietly(library(here))
quietly(library(edgeR))

In [289]:
select_probes = function(gset, gene_col_name) {
    
    keep_probes = as.data.frame(exprs(gset)) %>%
    
        # compute total expression across samples per probe ID
        mutate(probe_sum=rowSums(.)) %>%
        rownames_to_column('probe_id') %>%
    
        # join expression data to gene symbols associated with each probe ID
        left_join(
            rownames_to_column(fData(gset), 'probe_id') %>%
                rename(gene_symbol = all_of(gene_col_name)) %>%
                select(probe_id, gene_symbol), 
            by='probe_id') %>%
    
        # for probe IDs mapping to multiple genes, explode gene symbols to get 1 per row 
        separate_rows(gene_symbol, sep='///') %>%
        extract(gene_symbol, into=c('gene_symbol', 'gene_symbol1'),
                regex='^[\\s]*([[:graph:]]+)\\s//\\s([[:graph:]]+)') %>%
        mutate(gene_symbol = ifelse(is.na(gene_symbol1), gene_symbol, gene_symbol1)) %>%
        select(-gene_symbol1) %>%
        mutate(gene_symbol = str_trim(gene_symbol)) %>%
        mutate(gene_symbol = na_if(gene_symbol, '')) %>%
        drop_na(gene_symbol) %>%
        
        # for each gene, select probe ID with greatest total expression across samples
        group_by(gene_symbol) %>%
        top_n(1, probe_sum) %>%
        ungroup() %>%
    
        # remove duplicate probe IDs (probes mapping to more than one gene)
        distinct(probe_id) %>%
        pull()
    
    return(keep_probes)
}

In [274]:
is_log_normalized = function(gset) {

    exprs = exprs(gset)
    
    # non-missing, variant columns
    cols = apply(exprs, 2, function(x) sd(x, na.rm=TRUE) > 0)
    
    # test using first column
    test_vec = exprs[,cols][,1]
                 
    # add offset to make all values positive, if necessary
    min_value = min(test_vec, na.rm=TRUE)
    if (min_value <= 0) {
        test_vec = test_vec - min_value + 1
    }
                 
    real = qqnorm(test_vec, plot.it=FALSE)
    exp = qqnorm(exp(test_vec), plot.it=FALSE)
    log = qqnorm(log(test_vec), plot.it=FALSE)
                 
    exp$x[which(is.infinite(exp$x))] = NA
    exp$y[which(is.infinite(exp$y))] = NA

    cor_real = cor(real$x, real$y, use='pairwise.complete.obs')
    cor_exp = cor(exp$x, exp$y, use='pairwise.complete.obs')
    cor_log = cor(log$x, log$y, use='pairwise.complete.obs')  
    
    if (all(is.na(cor_exp))) {
        score = cor_log - cor_real
    } else {
        score = log2(abs(cor_real**2 - cor_log**2)/abs(cor_real**2 - cor_exp**2))
    }
        
    return(score <= 0)

}

In [415]:
quantile_normalize = function(p, exprs, sample_phenotypes) {
    samples = names(Filter(function(x) { x == p }, sample_phenotypes))
    exprs_norm = preprocessCore::normalize.quantiles(
        as.matrix(exprs[,samples])) %>% data.frame
    rownames(exprs_norm) = rownames(exprs)
    colnames(exprs_norm) = samples
    return(exprs_norm)
}

In [416]:
code_phenotype = function(gsm_id, phenotypes, control, case) {
    if (phenotypes[[gsm_id]] == control) { return(0) }
    else if (phenotypes[[gsm_id]] == case) { return(1) }
    else { return(NA) }
}

In [417]:
fit_differential_expression_model = function(comparison, exprs, dataset, data_dir) {

    coded_phenotypes = sapply(
        colnames(exprs), code_phenotype, dataset$phenotypes,
        comparison$control, comparison$case)

    coded_phenotypes = as.factor(coded_phenotypes[!is.na(coded_phenotypes)])

    if (length(unique(coded_phenotypes)) == 2) { 
 
        design_matrix = model.matrix(~0 + coded_phenotypes)
        rownames(design_matrix) = names(coded_phenotypes)[!is.na(coded_phenotypes)]
        colnames(design_matrix) = c('control', 'case')
        exprs = exprs[,rownames(design_matrix)]
        
        if (dataset$platform == 'rnaseq') {
            
            # filter out genes that don't have at least 1 count-per-million in at least half the samples
            exprs = exprs[rowSums(cpm(exprs)) >= (ncol(exprs)/2),]
            dge_list = DGEList(
                counts=exprs, group=coded_phenotypes, genes=rownames(exprs))
            norm_factors = calcNormFactors(dge_list)
            exprs = voom(dge_list, design_matrix)
           
        } 

        limma_fit = lmFit(exprs, design_matrix)
        contrast_matrix = makeContrasts(case - control, levels=design_matrix)
        contrast_fit = contrasts.fit(limma_fit, contrast_matrix)
        contrast_fit_ebayes = eBayes(contrast_fit, proportion=0.01)
        
        results_df = topTable(
            contrast_fit_ebayes, genelist=rownames(exprs),
            adjust='fdr', sort='none', n=Inf)

        results_df = results_df %>%
            rename(gene_symbol='ID',
                   log_fc=logFC,
                   adj_p_val=adj.P.Val) %>%
            mutate(dataset=dataset$gse_id,
                   control=comparison$control,
                   case=comparison$case) %>%
            select(dataset,
                   control,
                   case,
                   gene_symbol,
                   log_fc,
                   adj_p_val) 
        
        return(results_df)
    }
}

In [418]:
differential_expression_analysis = function(dataset, comparisons, platforms, data_dir, qnorm=TRUE) {
    
    # process microarray datasets
    if (startsWith(dataset$platform, 'GPL')) {

        # fetch dataset from GEO
        gset = quietly(getGEO(dataset$gse_id, AnnotGPL=TRUE))
        if (length(gset) > 1) {
            idx = grep(dataset$platform, attr(gset, 'names'))
        } else {
            idx = 1
        }
        gset = gset[[idx]]
        
        # log normalize expression matrix values if not already
        if (!is_log_normalized(gset)) {
            
            # values may still be negative due to background subtraction algo, add offset
            min_val = min(exprs(gset), na.rm=TRUE)
            if (min_val < 0) {
                exprs(gset) = exprs(gset) - min_val + 1
            }
            
            # log2 normalize
            exprs(gset) = log2(exprs(gset))
        }
        
        # format feature annotation names
        fvarLabels(gset) = make.names(fvarLabels(gset))
        
        # name of gene symbol column varies by microarray platform
        gene_col_name = platforms[[dataset$platform]]$gene_col_name

        # create expression dataframe
        exprs_df = fData(gset) %>%
            rename(gene_symbol = all_of(gene_col_name)) %>%
            select(gene_symbol) %>%
            rownames_to_column('probe_id') %>%
        
            # join gene symbol to expression matrix
            inner_join(
                exprs(gset) %>%
                    data.frame %>%
                    rownames_to_column('probe_id'),
                by='probe_id') %>%
            select(-probe_id) %>%
        
            # compute total expression per probe ID across samples
            mutate(probe_sum = rowSums(across(where(is.numeric)))) %>%
        
            # explode rows with multiple gene symbols
            separate_rows(gene_symbol, sep='///') %>%
            mutate(gene_symbol = str_trim(gene_symbol)) %>%
        
            # drop missing gene symbols
            mutate(gene_symbol = na_if(gene_symbol, '')) %>%
            mutate(gene_symbol = na_if(gene_symbol, '---')) %>%
            mutate(gene_symbol = na_if(gene_symbol, 'previous version conserved probe')) %>%
            drop_na(gene_symbol) %>%
        
            # parse out gene symbol
            extract(gene_symbol, into=c('gene_symbol', 'gene_symbol1'),
                    regex='^[\\s]*([[:graph:]]+)[\\s//\\s]*([[:graph:]]+)*') %>%
            mutate(gene_symbol1 = na_if(gene_symbol1, '')) %>%
            mutate(gene_symbol = ifelse(is.na(gene_symbol1), gene_symbol, gene_symbol1)) %>%
            select(-gene_symbol1) %>%
        
            # explode rows with multiple gene symbols (again)
            separate_rows(gene_symbol, sep='\\|') %>%
            mutate(gene_symbol = str_trim(gene_symbol)) %>%
        
            # keep one row per gene symbol (row with highest expression across samples)
            group_by(gene_symbol) %>%
            slice_max(n=1, order_by=probe_sum, with_ties=FALSE) %>%
            ungroup() %>%
            select(-probe_sum) %>%
            column_to_rownames('gene_symbol')
        
        # remove null phenotype samples
        is_null = sapply(dataset$phenotypes, is.null)
        keep_samples = intersect(
            names(is_null)[!is_null], colnames(exprs_df))
        dataset$phenotypes = dataset$phenotypes[keep_samples]
        exprs_df = select(exprs_df, all_of(keep_samples))
        
        # set of unique phenotypes in the dataset
        phenotypes = unique(dataset$phenotypes)
        
        # quantile normalize within phenotype groups
        norm_dfs = lapply(phenotypes, quantile_normalize, exprs_df, dataset$phenotypes)
        exprs = as.matrix(bind_cols(norm_dfs))
    }
    
    # process rnaseq datasets
    else if (dataset$platform == 'rnaseq') {
        
        filename = paste(
            dataset$gse_id, 'salmon', 'genes', 'read_count',
            'tximport', 'tsv', sep='.')
        filepath = file.path(data_dir, 'rnaseq-datasets', filename)
        
        df_exprs = read.table(filepath, sep='\t', header=TRUE)
        df_exprs = inner_join(gene_id_symbol_map, df_exprs, by='gene_id') %>%
            select(-gene_id)
        colnames(df_exprs)[2:ncol(df_exprs)] = sapply(
            colnames(df_exprs)[2:ncol(df_exprs)], function(x) srr_gsm_id_map[[x, 'gsm_id']])
        
        # for each gene symbol, keep only row with highest total expression across samples
        df_exprs = df_exprs %>%
            group_by(gene_symbol) %>%
            arrange(desc(rowSums(across(where(is.numeric))))) %>%
            filter(row_number() == 1) %>%
            column_to_rownames('gene_symbol')
            
        # remove null phenotype samples
        is_null = sapply(dataset$phenotypes, is.null)
        keep_samples = intersect(
            names(is_null)[!is_null], colnames(df_exprs))
        dataset$phenotypes = dataset$phenotypes[keep_samples]
        df_exprs = df_exprs %>% select(all_of(keep_samples))
        
        exprs = as.matrix(df_exprs) 
    }
            
    dataset_results_list = lapply(
        comparisons, fit_differential_expression_model,
        exprs, dataset, file.path(data_dir))
            
    dataset_results_df = bind_rows(dataset_results_list)
    rownames(dataset_results_df) = NULL
            
    return(dataset_results_df)
}

In [419]:
datasets = fromJSON(file=here('data', 'datasets.json'))

In [420]:
platforms = fromJSON(file=here('data', 'platforms.json'))

In [421]:
gene_id_symbol_map = read.table(
    here('data', 'gene_id_symbol_map.tsv'), sep='\t', header=TRUE)

In [422]:
srr_gsm_id_map = read.table(
    here('data', 'srr_gsm_id_map.tsv'), sep='\t', header=TRUE) %>% 
    column_to_rownames('srr_id')

In [423]:
comparisons = fromJSON(file=here('data', 'comparisons.json'))

In [424]:
data_dir = file.path('/efs/liam/tb-gene-signature-datasets/network-analysis/convert-to-scripts')
quietly(dir.create(data_dir))

In [411]:
datasets = Filter(function(x) x$gse_id == 'GSE41055', datasets)
#datasets = Filter(function(x) x$gse_id == 'GSE19442', datasets)
#datasets = Filter(function(x) x$gse_id == 'GSE62525', datasets)

In [412]:
dataset = datasets[[1]]
comparison = comparisons[[1]]

In [413]:
# fetch dataset from GEO
gset = quietly(getGEO(dataset$gse_id, AnnotGPL=TRUE))
if (length(gset) > 1) {
    idx = grep(dataset$platform, attr(gset, 'names'))
} else {
    idx = 1
}
gset = gset[[idx]]

# format feature annotation names
fvarLabels(gset) = make.names(fvarLabels(gset))

In [414]:
gene_col_name = platforms[[dataset$platform]]$gene_col_name

inner_join(
    fData(gset) %>% rename(gene_symbol=all_of(gene_col_name)) %>% select(gene_symbol) %>% rownames_to_column('probe_id'),
    exprs(gset) %>% data.frame %>% rownames_to_column('probe_id'),
    by='probe_id') %>%
select(-probe_id) %>%
mutate(probe_sum=rowSums(across(where(is.numeric)))) %>%
separate_rows(gene_symbol, sep='///') %>%
mutate(gene_symbol = str_trim(gene_symbol)) %>%
mutate(gene_symbol = na_if(gene_symbol, '')) %>%
mutate(gene_symbol = na_if(gene_symbol, '---')) %>%
mutate(gene_symbol = na_if(gene_symbol, 'previous version conserved probe')) %>%
drop_na(gene_symbol) %>%
extract(gene_symbol, into=c('gene_symbol', 'gene_symbol1'),
        regex='^[\\s]*([[:graph:]]+)[\\s//\\s]*([[:graph:]]+)*') %>%
mutate(gene_symbol1 = na_if(gene_symbol1, '')) %>%
mutate(gene_symbol = ifelse(is.na(gene_symbol1), gene_symbol, gene_symbol1)) %>%
select(-gene_symbol1) %>%
separate_rows(gene_symbol, sep='\\|') %>%
mutate(gene_symbol = str_trim(gene_symbol)) %>%
group_by(gene_symbol) %>%
slice_max(n=1, order_by=probe_sum, with_ties=FALSE) %>%
ungroup() %>%
select(-probe_sum) %>%
column_to_rownames('gene_symbol')

,GSM1008060,GSM1008061,GSM1008062,GSM1008063,GSM1008064,GSM1008065,GSM1008066,GSM1008067,GSM1008068,GSM1008069,⋯,GSM1008077,GSM1008078,GSM1008079,GSM1008080,GSM1008081,GSM1008082,GSM1008083,GSM1008084,GSM1008085,GSM1008086
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,147.43078,141.12619,142.80231,132.29856,125.32796,133.61265,109.63957,129.19810,135.25415,135.97625,⋯,143.63387,123.00410,145.99362,140.80232,129.70300,133.67078,124.06644,124.53123,140.86029,124.60901
A1BG-AS,147.43078,141.12619,142.80231,132.29856,125.32796,133.61265,109.63957,129.19810,135.25415,135.97625,⋯,143.63387,123.00410,145.99362,140.80232,129.70300,133.67078,124.06644,124.53123,140.86029,124.60901
A1CF,40.53607,36.00159,46.28471,40.86220,40.41743,46.90629,38.69738,42.57817,49.20452,38.20887,⋯,51.32248,37.98662,48.96804,44.45799,38.40374,41.94511,44.16201,43.97312,43.57345,43.41485
A2LD1,89.17131,81.34287,108.30285,69.67720,66.34622,69.09363,80.37890,73.06804,88.73285,85.74047,⋯,62.05834,100.97897,81.64952,82.83682,76.50288,75.25542,67.58936,63.27835,73.49291,61.54749
A2M,60.60614,61.56835,56.28508,63.22362,46.02319,67.49238,60.42353,49.78100,66.18785,59.72361,⋯,61.40453,57.95094,55.02447,54.84805,57.44656,54.53207,57.52597,55.39453,67.56236,56.27940
A2ML1,19.67550,21.63538,17.57986,21.85792,23.94658,19.07748,16.60913,21.34770,22.27395,22.10500,⋯,24.04092,28.20141,26.08720,25.56043,21.73657,19.09019,24.26079,18.29847,18.61503,25.81995
A4GALT,65.68304,72.36281,72.25317,53.60154,61.01406,71.17502,46.56974,54.66147,72.88231,59.72487,⋯,56.63430,56.85706,88.91965,60.37676,46.65756,72.66836,68.30944,52.56120,58.48239,65.57801
A4GNT,57.53200,59.88787,65.29267,57.73387,55.39038,59.87562,61.31953,53.73023,60.42699,76.51055,⋯,50.64196,60.97198,76.71797,59.73387,55.10075,52.65602,57.54183,42.72785,51.83020,59.02105
AAA1,40.10540,43.77121,42.79321,33.24103,37.12641,35.05682,36.40548,36.73238,46.15441,41.64144,⋯,36.94945,35.82664,45.91766,36.48982,32.95608,41.89649,32.31278,31.49515,41.24263,40.65432


In [404]:
# filter probes (rows in expression matrix)
keep_probes = select_probes(
    gset, platforms[[dataset$platform]]$gene_col_name)
gset = gset[keep_probes,]

# remove null phenotype samples
is_null = sapply(dataset$phenotypes, is.null)
keep_samples = intersect(
    names(is_null)[!is_null], colnames(exprs(gset)))
dataset$phenotypes = dataset$phenotypes[keep_samples]
gset = gset[,keep_samples]

In [425]:
all_results_list = lapply(datasets, differential_expression_analysis, comparisons, platforms, data_dir)

Note: Using an external vector in selections is ambiguous.
i Use `all_of(keep_samples)` instead of `keep_samples` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [426]:
all_results_df = bind_rows(all_results_list)

In [438]:
all_results_df[all_results_df$gene_symbol=='LOC100499484-C9ORF174',]

,dataset,control,case,gene_symbol,log_fc,adj_p_val
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
581337,GSE54992,hc,atb,LOC100499484-C9ORF174,0.593110572,6.056960e-02
604173,GSE54992,hc,ltbi,LOC100499484-C9ORF174,-0.363622267,9.999084e-01
627009,GSE54992,ltbi,atb,LOC100499484-C9ORF174,0.956732838,1.607340e-02
686342,GSE62525,hc,atb,LOC100499484-C9ORF174,0.003154744,9.696894e-01
707012,GSE62525,hc,ltbi,LOC100499484-C9ORF174,0.110719946,2.633255e-01
727682,GSE62525,ltbi,atb,LOC100499484-C9ORF174,-0.107565202,3.636625e-01
769002,GSE73408,od,atb,LOC100499484-C9ORF174,0.158648909,1.506784e-01
789791,GSE73408,ltbi,atb,LOC100499484-C9ORF174,-0.465628296,1.107156e-07


In [437]:
all_results_df

dataset,control,case,gene_symbol,log_fc,adj_p_val
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
GSE19439,hc,atb,A1BG,2.604504e-02,0.9172166
GSE19439,hc,atb,A1CF,9.143472e-02,0.4683955
GSE19439,hc,atb,A2M,7.355991e-03,0.9646348
GSE19439,hc,atb,A2ML1,-1.710588e-01,0.2044012
GSE19439,hc,atb,A3GALT2,-8.727415e-03,0.9673175
GSE19439,hc,atb,A4GALT,8.690103e-02,0.6328155
GSE19439,hc,atb,A4GNT,3.208599e-02,0.8212644
GSE19439,hc,atb,AA06,-2.273128e-05,0.9999200
GSE19439,hc,atb,AAAS,-1.050407e-01,0.4072695


In [ ]:
dataset = datasets[[13]]

In [ ]:
# fetch dataset from GEO
gset = quietly(getGEO(dataset$gse_id, AnnotGPL=TRUE))
if (length(gset) > 1) {
    idx = grep(dataset$platform, attr(gset, 'names'))
} else {
    idx = 1
}
gset = gset[[idx]]

# format feature annotation names
fvarLabels(gset) = make.names(fvarLabels(gset))

# filter probes (rows in expression matrix)
keep_probes = select_probes(
    gset, platforms[[dataset$platform]]$gene_col_name)
gset = gset[keep_probes,]

# remove null phenotype samples
is_null = sapply(dataset$phenotypes, is.null)
keep_samples = intersect(
    names(is_null)[!is_null], colnames(exprs(gset)))
dataset$phenotypes = dataset$phenotypes[keep_samples]
gset = gset[,keep_samples]

# log normalize if not already
if (!is_log_normalized(gset)) {

    # values may still be negative due to background subtraction algo, add offset
    min_val = min(exprs(gset), na.rm=TRUE)
    if (min_val < 0) {
        exprs(gset) = exprs(gset) - min_val + 1
    }

    # log2 normalize
    exprs(gset) = log2(exprs(gset))
}

In [ ]:
design_matrix = model.matrix(~0 + coded_phenotypes)
rownames(design_matrix) = names(coded_phenotypes)[!is.na(coded_phenotypes)]
colnames(design_matrix) = c('control', 'case')
exprs = exprs[,rownames(design_matrix)]

In [ ]:
limma_fit = lmFit(exprs, design_matrix)
contrast_matrix = makeContrasts(case - control, levels=design_matrix)
contrast_fit = contrasts.fit(limma_fit, contrast_matrix)
contrast_fit_ebayes = eBayes(contrast_fit, proportion=0.01)

In [ ]:
results_table1 = topTable(
        contrast_fit_ebayes, genelist=rownames(exprs),
        adjust='fdr', sort='none', n=Inf) %>%
    rename(gene_symbol='ID') %>%
    select(gene_symbol, logFC, adj.P.Val)

In [ ]:
results_table